## Preprocessing
Unlike LDA we should not toy too much with the sentence structure as ABAE uses word embeddings and needs the sequence information to weight the terms based on the surrounding context. One question remains:

**Should we work on sentence level or full reviews? Let's try a first simple comparison**

### Full-reviews

In [21]:
from main.abae.model_manager import ABAEManagerConfig, ABAEManager

corpus = "../dataset/output/default/pre_processed.80k.csv"
default_config = ABAEManagerConfig(model_name="abae_default_ds")
abae_manager = ABAEManager.from_scratch(default_config, corpus)

Pandas Apply:   0%|          | 0/60640 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae_default_ds\abae_default_ds.embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae_default_ds\\abae_default_ds.embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae_default_ds\abae_default_ds.embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae_default_ds\\abae_default_ds.embeddings.model', 'datetime': '2025-03-11T22:11:14.085815', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


Loading the existing found model as requested in path ./output/abae_default_ds/abae_default_ds.aspect_embeddings.model


In [ ]:
history, _ = abae_manager.train(corpus)
model = abae_manager.get_compiled_model(refresh=False)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/60640 [00:00<?, ?it/s]

Max sequence length calculation in progress...


Latest run result:
```
Max Margin loss: [4.6614, 4.6604]
```

In [3]:
from evaluation import ABAEEvaluationProcessor

inv_vocab = abae_manager.generator.emb_model.model.wv.index_to_key
processor = ABAEEvaluationProcessor.generate_for_model(model, inv_vocab)

In [15]:
from main.abae.dataset import PositiveNegativeABAEDataset
import pandas as pd
from torch.utils.data import DataLoader

test_corpus_path = "../dataset/output/default/pre_processed.80k.test.csv"
df = pd.read_csv(test_corpus_path)

npmi_coh = processor.c_npmi_coherence_model(top_n=10, ds=df['comments'].apply(lambda x: x.split(' ')))
npmi_coherence = npmi_coh.get_coherence()

cv_coh = processor.c_v_coherence_model(top_n=100, ds=df['comments'].apply(lambda x: x.split(' ')))
cv_coherence = cv_coh.get_coherence()

vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
test_ds = PositiveNegativeABAEDataset(df, vocabulary, max_seq_len, negative_sample_size)

res = model.evaluate(DataLoader(test_ds, batch_size=default_config.batch_size))
print(f"Max margin reconstruction result: {res}")

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<12053 unique tokens: ['<game_name>', 'abstract', 'add', 'art', 'bad']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<16753 unique tokens: ['<game_name>', 'abstract', 'add', 'art', 'bad']...>
INFO:gensim.corpora.dictionary:built Dictionary<16850 unique tokens: ['<game_name>', 'abstract', 'add', 'art', 'bad']...> from 20213 documents (total 390827 corpus positions)
DEBUG:gensim.utils:starting a new internal lifecycle event log for Dictionary
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<16850 unique tokens: ['<game_name>', 'abstract', 'add', 'art', 'bad']...> from 20213 documents (total 390827 corpus positions)", 'datetime': '2025-03-10T01:00:04.936508', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/20213 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 464(2.295552367288379% of ds).
158/158 ━━━━━━━━━━━━━━━━━━━━ 19s 120ms/step - loss: 4.7868 - max_margin_loss: 4.7858
Max margin reconstruction result: [4.745960235595703, 4.744936943054199]


In [16]:
print(f"NPMI coherence: {npmi_coherence}")
print(f"CV score: {cv_coherence}")
print(f"Max margin reconstruction result: {res}")

NPMI coherence: -0.23037988688672537
CV score: 0.5646751917101251
Max margin reconstruction result: [4.745960235595703, 4.744936943054199]


Results of the latest run:
```
NPMI coherence: -0.23037988688672537
CV score: 0.5646751917101251
Max margin reconstruction result: [4.745960235595703, 4.744936943054199]
```

In [22]:
list(processor.extract_top_k_words(10, 10))

[('instruction', tensor(0.6540, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('manual', tensor(0.6516, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('page', tensor(0.6307, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('solution', tensor(0.6230, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('reference', tensor(0.5907, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('query', tensor(0.5859, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('vague', tensor(0.5815, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('wiki', tensor(0.5806, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('decipher', tensor(0.5799, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('rulebook', tensor(0.5764, device='cuda:0', grad_fn=<SelectBackward0>))]

### Sentence-split reviews

In [1]:
from main.abae.model_manager import ABAEManagerConfig, ABAEManager

corpus = "../dataset/output/default_sentences/pre_processed.80k.csv"
default_config = ABAEManagerConfig(model_name="abae_sent_ds")
abae_manager = ABAEManager.from_scratch(default_config, corpus, False)

Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae_sent_ds\abae_sent_ds.embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae_sent_ds\\abae_sent_ds.embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae_sent_ds\abae_sent_ds.embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae_sent_ds\\abae_sent_ds.embeddings.model', 'datetime': '2025-03-11T22:11:28.603814', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


Loading the existing found model as requested in path ./output/abae_sent_ds/abae_sent_ds.aspect_embeddings.model


In [2]:
abae_manager

In [ ]:
history, _ = abae_manager.train(corpus)
model = abae_manager.get_compiled_model(refresh=False)

In [26]:
# print history
history.history

{'loss': [8.786524772644043,
  5.993138313293457,
  5.584963321685791,
  5.477548122406006,
  5.42295503616333,
  5.371152877807617,
  5.343845844268799,
  5.310521125793457,
  5.257896423339844,
  5.230459690093994,
  5.210410118103027,
  5.190746307373047,
  5.181276321411133,
  5.164516448974609,
  5.154855728149414],
 'max_margin_loss': [8.780280113220215,
  5.991584777832031,
  5.584194660186768,
  5.476739883422852,
  5.423596382141113,
  5.369997978210449,
  5.3439788818359375,
  5.3094024658203125,
  5.2574005126953125,
  5.22948694229126,
  5.208757400512695,
  5.1898369789123535,
  5.179889678955078,
  5.1640849113464355,
  5.154535293579102]}

In [2]:
model = abae_manager.get_inference_model(refresh=True)

In [27]:
model.get_layer('emb')

ValueError: No such layer: emb. Existing layers are: ['positive', 'word_embedding', 'attention', 'weight', 'sentence_aspect'].

In [7]:
b.dataset.tolist()

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  274,
  413,
  808,
  226,
  556],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  129,
  1684,
  1,
  1117,
  5758],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [6]:
b[1][0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  129, 1684,
          1, 1117, 5758])

In [8]:
from main.abae.dataset import ABAEDataset, PositiveNegativeABAEDataset
from torch.utils.data import DataLoader
import numpy as np

vocabulary = abae_manager.generator.emb_model.vocabulary()
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"
b = ABAEDataset(test_corpus_path, vocabulary, 80)
pred = model.predict(DataLoader(b))
pred

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/22796 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 2(0.008773469029654327% of ds).
22795/22795 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step


[array([[0.        , 0.        , 0.        , ..., 0.1548343 , 0.15425645,
         0.22813106],
        [0.        , 0.        , 0.        , ..., 0.05024783, 0.3206124 ,
         0.09269985],
        [0.        , 0.        , 0.        , ..., 0.02223487, 0.02223487,
         0.02223487],
        ...,
        [0.        , 0.        , 0.        , ..., 0.04031339, 0.29732263,
         0.27889398],
        [0.        , 0.        , 0.        , ..., 0.03535273, 0.26122296,
         0.20121163],
        [0.        , 0.        , 0.        , ..., 0.10950671, 0.07589232,
         0.03285409]], dtype=float32),
 array([[0.06032778, 0.04643534, 0.09291431, ..., 0.06347105, 0.06767754,
         0.06360632],
        [0.05167519, 0.05393037, 0.08228585, ..., 0.10430072, 0.03749444,
         0.07167535],
        [0.08552562, 0.0526678 , 0.05749592, ..., 0.06339459, 0.08560044,
         0.06950204],
        ...,
        [0.0739431 , 0.04744409, 0.07656203, ..., 0.07479972, 0.08239473,
         0.0543629 

In [33]:
aspect_embeddings.shape

(14, 100)

In [55]:
np.stack(b.dataset.map(lambda x: np.array(x)).to_numpy())

array([[   0,    0,    0, ...,  808,  226,  556],
       [   0,    0,    0, ...,    1, 1117, 5758],
       [   0,    0,    0, ...,   99,  483,  106],
       ...,
       [   0,    0,    0, ...,    1, 1736, 2280],
       [   0,    0,    0, ..., 3997, 3296,  158],
       [   0,    0,    0, ...,   90,  345,   48]])

In [74]:
aspect_embeddings[0][-1]

tensor([-0.1865,  1.1079, -0.2665,  0.2179, -0.6209, -0.5075, -0.7768,  1.3716,
         1.3456,  0.0593,  1.1050, -0.2196,  0.3597,  0.8171, -2.0318,  0.2083,
         1.1928, -0.4836, -0.3990,  0.3118,  0.1351, -0.5598,  0.1043,  0.0579,
         0.2841,  0.1915, -0.2961, -1.1808,  0.7000,  0.6386, -0.2225,  0.7734,
         0.2675,  1.7498,  0.5138,  0.0733, -0.6502, -0.4926,  0.2966, -0.0413,
         0.3259,  0.5075,  1.0663, -1.2625, -0.6104,  0.5915,  0.1661, -0.3879,
        -1.9599,  0.1427,  0.5113, -0.8907, -1.1449,  0.2808,  0.7434,  0.1067,
        -0.9035,  0.8058, -0.3569,  0.6006,  0.0333, -2.1432,  0.2018, -0.4316,
         0.5455, -1.0774,  0.2203,  0.1884,  0.4217, -0.4151,  0.1997, -0.5199,
        -0.4074, -1.7790, -0.3785, -0.3534, -1.1516, -0.1487, -1.2788,  0.9017,
        -1.8429, -0.0913,  1.5070, -1.0603,  1.3809, -1.2154,  0.2412, -0.0239,
         0.2560,  0.9579, -1.4903,  0.5514,  0.1526,  1.7090,  1.2487,  0.0190,
        -1.3630,  0.5898,  1.4117, -0.08

In [84]:
w_embs[0][-1]

tensor(-0.2824, device='cuda:0')

TypeError: 'KeyedVectors' object is not callable

In [100]:
n = [w[w != 0] for w in w_embs]

In [97]:
w_embs[0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.2706, -1.0092, -0.1415,  1.1576,  0.5602, -0.2824])

In [98]:
w_embs[0][w_embs[0] != 0]

tensor([ 0.2706, -1.0092, -0.1415,  1.1576,  0.5602, -0.2824])

In [138]:
n_em = []
for emb, att in zip(embeddings, attention):
    w = (att[..., np.newaxis] * emb.numpy()).sum(0)
    # w = Weight()([att, emb.cuda() ])
    n_em.append(w)
n_em[0]

array([-3.01971659e-02,  4.54186082e-01, -2.70544797e-01,  3.93439353e-01,
       -2.58552492e-01, -1.77031681e-01, -6.65187389e-02,  4.32498187e-01,
        2.73840755e-01,  1.44470990e-01,  1.44108966e-01,  1.15044042e-01,
        1.64253891e-01,  1.34186298e-02, -5.44844866e-01, -3.82557988e-01,
        1.06261754e+00, -9.99008954e-01, -3.52798939e-01,  3.25612277e-01,
        1.27736405e-02, -2.83427119e-01,  2.88097173e-01,  4.44390893e-01,
        3.29987556e-01,  5.69236338e-01, -9.96254236e-02, -1.71565950e-01,
        2.14732781e-01,  7.27142990e-02,  2.42562965e-02,  3.05961549e-01,
        7.70513654e-01,  3.32429528e-01,  8.86130929e-01, -4.06002492e-01,
       -5.74916840e-01,  1.71533674e-01,  4.89998087e-02, -3.65234256e-01,
        6.07235789e-01,  3.50003541e-01,  1.90377116e-01, -1.15104109e-01,
       -5.96605241e-02, -5.54661751e-01, -6.95270360e-01,  9.18035954e-03,
       -3.51217628e-01, -1.54343694e-02, -8.30099955e-02, -2.08590180e-01,
       -2.73632526e-01, -

In [105]:
n_em[0]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  1.1576,  1.1533,  1.7056],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5623,  0.5602,  0.8285],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1917, -0.1909, -0.2824]])

In [87]:
from sklearn.metrics import silhouette_score
from main.abae.layer import Weight

mm = abae_manager.get_compiled_model()

attention = pred[0]
embeddings = mm.get_layer(index=1)(np.stack(b.dataset.map(lambda x: np.array(x)).to_numpy())).cpu()

n_em = []
for emb, att in zip(embeddings, attention):
    w = (att[..., np.newaxis] * emb.numpy()).sum(0)
    # w = Weight()([att, emb.cuda() ])
    n_em.append(w)
n_em[0]

# Prediceted aspect

labels = np.argmax(pred[1], axis=1)

silhouette_score(w_embs.cpu(), labels)

AttributeError: 'list' object has no attribute 'cpu'

In [139]:
silhouette_score(n_em, labels, metric='cosine')  # -0.06 vs 0.10

0.103942364

In [141]:
run_result = dict()
# Questa va in caso perdi quella in processor come funzioni
vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
eval_ds = PositiveNegativeABAEDataset(test_corpus_path, vocabulary, max_seq_len, negative_sample_size)

model = abae_manager.get_compiled_model(refresh=False)
run_result['max_margin_loss'] = model.evaluate(DataLoader(eval_ds, batch_size=default_config.batch_size))
ev_model = abae_manager.get_inference_model()

pred_val_data = ABAEDataset(test_corpus_path, vocabulary, max_seq_len)
att, labels = ev_model.predict(DataLoader(pred_val_data, batch_size=default_config.batch_size, shuffle=False))

# todo verifica
# Calculate the embeddings of the model.
embeddings = model.get_layer(index=1)(np.stack(pred_val_data.dataset.map(lambda x: np.array(x))))
# Weight the embeddings to create sentence embeddings
w_embs = [(att[..., np.newaxis] * emb.cpu().numpy()).sum(0) for emb, att in zip(embeddings, att)]
run_result['silhouette_score'] = silhouette_score(w_embs, np.argmax(labels, axis=1))

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/22796 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 2(0.008773469029654327% of ds).
179/179 ━━━━━━━━━━━━━━━━━━━━ 22s 121ms/step - loss: 20.7814 - max_margin_loss: 20.7789
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/22796 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 2(0.008773469029654327% of ds).
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [143]:
run_result

{'max_margin_loss': [20.76984405517578, 20.759178161621094],
 'silhouette_score': 0.043248724}

In [3]:
from evaluation import ABAEEvaluationProcessor

inv_vocab = abae_manager.generator.emb_model.model.wv.index_to_key
processor = ABAEEvaluationProcessor.generate_for_model(model, inv_vocab)

AttributeError: 'Word2VecWrapper' object has no attribute 'generator'

In [30]:
from main.abae.dataset import PositiveNegativeABAEDataset
import pandas as pd
from torch.utils.data import DataLoader

test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"
df = pd.read_csv(test_corpus_path)

npmi_coh = processor.c_npmi_coherence_model(top_n=10, ds=df['comments'].apply(lambda x: x.split(' ')))
npmi_coherence = npmi_coh.get_coherence()

cv_coh = processor.c_v_coherence_model(top_n=100, ds=df['comments'].apply(lambda x: x.split(' ')))
cv_coherence = cv_coh.get_coherence()

vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
test_ds = PositiveNegativeABAEDataset(df, vocabulary, max_seq_len, negative_sample_size)

res = model.evaluate(DataLoader(test_ds, batch_size=default_config.batch_size))
print(f"Max margin reconstruction result: {res}")

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<7608 unique tokens: ['hide', 'like', 'multiple', 'possible', 'traitor']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<10781 unique tokens: ['hide', 'like', 'multiple', 'possible', 'traitor']...>
INFO:gensim.corpora.dictionary:built Dictionary<11509 unique tokens: ['hide', 'like', 'multiple', 'possible', 'traitor']...> from 22796 documents (total 176244 corpus positions)
DEBUG:gensim.utils:starting a new internal lifecycle event log for Dictionary
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<11509 unique tokens: ['hide', 'like', 'multiple', 'possible', 'traitor']...> from 22796 documents (total 176244 corpus positions)", 'datetime': '2025-03-10T17:18:46.204925', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/22796 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 2(0.008773469029654327% of ds).
179/179 ━━━━━━━━━━━━━━━━━━━━ 23s 126ms/step - loss: 5.2631 - max_margin_loss: 5.2623
Max margin reconstruction result: [5.261548042297363, 5.262136936187744]


In [31]:
print(f"NPMI coherence: {npmi_coherence}")
print(f"CV score: {cv_coherence}")
print(f"Max margin reconstruction result: {res}")

NPMI coherence: -0.3117940799781337
CV score: 0.6031670887985386
Max margin reconstruction result: [5.261548042297363, 5.262136936187744]


Results for the run:
```
NPMI coherence: -0.3117940799781337
CV score: 0.6031670887985386
Max margin reconstruction result: [5.261548042297363, 5.262136936187744]
```

In [37]:
list(processor.extract_top_k_words(11, 10))

[('instruction', tensor(0.7166, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('rule', tensor(0.7164, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('rulebook', tensor(0.7001, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('manual', tensor(0.6964, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('forum', tensor(0.6304, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('faq', tensor(0.6292, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('language', tensor(0.5814, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('booklet', tensor(0.5774, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('iconography', tensor(0.5654, device='cuda:0', grad_fn=<SelectBackward0>)),
 ('rules', tensor(0.5641, device='cuda:0', grad_fn=<SelectBackward0>))]

I know that doing a  comparison on a single run is not that meaningful. <br>
I could do k-CV to estimate the expected model loss to get a valid analysis. <br>
But for the sake of the experiment we consider this good enough.

For the future work and as one by the proposed ABAE paper we won't be splitting up reviews in sentences but use the full review as the model does not increase much if not done like this.